# 📊 **Zákaznický Příjmový a Rizikový Dashboard**

Vítejte na interaktivním dashboardu zaměřeném na analýzu příjmů a rizik klientského portfolia. Tento nástroj vám umožní nahlédnout do klíčových faktorů ovlivňujících příjmy podle produktů a rozložení rizikových kategorií, a také efektivně identifikovat a sledovat nejrizikovější klienty.

### 💡 **Hlavní funkce dashboardu:**

1. **Přehled Příjmů Podle Produktu**  
   - Vizualizace průměrného příjmu v jednotlivých produktových kategoriích.
   - Interaktivní výběr produktu pro detailní analýzu a přehled průměrného příjmu.

2. **Rizikový Profil a Doporučení**  
   - Analýza rozložení rizikových kategorií v klientském portfoliu.
   - Doporučení na základě aktuálního rizikového rozložení:
     - Upozornění na zvýšené riziko s doporučením, jak optimalizovat nabídku.
     - Návod na možné strategické kroky dle rozložení rizikového profilu.

3. **Interaktivní Výběr Rizikových Klientů**  
   - Možnost přehledného filtrování a výběru klientů s vysokým rizikem.
   - Funkce pro přidání poznámek k jednotlivým klientům přímo v aplikaci, což umožňuje efektivní sledování dalších kroků a komunikace s týmem.

### 🎯 **Cíl Dashboardu**

Tento dashboard vám poskytne nástroje a analytické vhledy potřebné pro rychlé a efektivní rozhodování o optimalizaci portfolia produktů, lepší kontrolu nad expozicí vůči klientům s vysokým rizikem a přehledné sledování klíčových klientů. Díky interaktivním možnostem si můžete přizpůsobit pohled na data dle aktuálních potřeb a obchodních priorit.

### 📌 **Návod k použití**

- **Výběr Produktu**: Zvolte produkt z rozbalovacího menu pro zobrazení detailních statistik.
- **Nastavení Rizikového Scénáře**: Úpravou rizikového faktoru můžete simulovat různé scénáře dopadu na příjem.
- **Přidání Poznámek k Rizikovým Klientům**: Vyberte klienta, zadejte poznámku a uložte ji, aby byly klíčové informace snadno dostupné vašemu týmu.

**Prozkoumejte možnosti dashboardu a získejte nový vhled do struktury příjmů a rizik vašeho klientského portfolia.**


In [ ]:
-- Výběr dat pro BI dashboard
SELECT 
    CLIENT_ID,
    AGE,
    INCOME,
    REGION,
    PRODUCT_TYPE,
    CREDIT_SCORE,
    PREDICTED_REVENUE,
    RISK_CATEGORY,
    SCENARIO_FACTOR,
    NOTE
FROM CUSTOMER_DATA;



In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
from snowflake.snowpark import Session  # Importujte Snowflake Session pro práci s databází

# Nastavení připojení ke Snowflake
session = Session.builder.getOrCreate()  # Vytvoří nebo načte existující Snowflake session

# Načtení výsledků SQL dotazu do Pandas DataFrame
df = session.sql("SELECT * FROM TEST.PUBLIC.CUSTOMER_DATA").to_pandas()  # Přímo načte data z tabulky

# Základní nastavení aplikace
st.title("Zákaznický Příjmový a Rizikový Dashboard")

# 1. Přehled rizikových kategorií a příjmů
st.subheader("Přehled Rizikových Kategorií a Příjmů Podle Produktu")

# Výběr produktu pro podrobnosti
selected_product = st.selectbox("Vyber produkt:", df['PRODUCT_TYPE'].unique())
filtered_df = df[df['PRODUCT_TYPE'] == selected_product]

# Zobrazení průměrného příjmu a rozložení rizikových kategorií
avg_income = filtered_df['INCOME'].mean()
risk_distribution = filtered_df['RISK_CATEGORY'].value_counts()

st.write(f"**Průměrný příjem pro produkt {selected_product}:** {avg_income:.2f} CZK")
st.write("**Rozložení rizikových kategorií:**")
st.bar_chart(risk_distribution)

# Doporučení na základě rizikového faktoru
st.subheader("Doporučení na základě Rizikového Rozložení")

# Přehled klientů s nejvyšším rizikem
st.write("Top 10 klientů s nejvyšším rizikem:")
high_risk_clients = df[df['RISK_CATEGORY'] == 'High Risk'].sort_values(by='CREDIT_SCORE').head(10)
st.dataframe(high_risk_clients[['CLIENT_ID', 'CREDIT_SCORE', 'INCOME', 'PRODUCT_TYPE']])

# Doporučení na základě rozložení rizikových kategorií
if risk_distribution.get("High Risk", 0) > 10:
    st.warning("Doporučujeme zvážit omezení expozice vůči klientům s vysokým rizikem.")
elif risk_distribution.get("Medium Risk", 0) > 10:
    st.info("Pozor na klienty s průměrným rizikem – doporučujeme přezkoumat podmínky produktů.")
else:
    st.success("Portfolio klientů je v nízkém riziku, pokračujte v současné strategii.")

# 2. Přidání poznámky k rizikovým klientům
st.subheader("Přidání Poznámky k Rizikovým Klientům")

# Možnost přidat poznámku k vybranému klientovi
selected_client = st.selectbox("Vyberte klienta pro přidání poznámky:", high_risk_clients['CLIENT_ID'])
note = st.text_input("Poznámka pro klienta", "Vyžaduje další posouzení")

# Uložení poznámky do databáze
if st.button("Uložit poznámku"):
    # SQL příkaz pro zápis poznámky k vybranému klientovi
    query = f"""
    UPDATE TEST.PUBLIC.CUSTOMER_DATA
    SET NOTE = '{note}'
    WHERE CLIENT_ID = '{selected_client}';
    """
    
    # Provedení SQL dotazu přímo z aplikace Streamlit
    session.sql(query).collect()
    
    st.success(f"Poznámka '{note}' byla uložena pro klienta {selected_client}.")


# 💡 Interaktivní Analýza Recenzí České Spořitelny

Vítejte v ukázce datové analýzy a vizualizace s využitím **Streamlit**, který umožňuje vytvářet interaktivní aplikace přímo v Pythonu.

## 🔎 Cíl Ukázky

**Jaký je hlavní cíl?** Analyzovat zpětnou vazbu klientů České spořitelny na pobočky pomocí moderních datových metod, které nám pomohou pochopit sentiment a klíčové oblasti, ve kterých by se služby mohly zlepšit.

Tato ukázka:
- 📊 Analyzuje sentiment jednotlivých recenzí klientů
- 🧠 Automaticky shrnuje klíčové body každé recenze pomocí **AI modelů**
- 🖋 Extrahuje klíčová slova, která napovídají o hlavních tématech

## 🚀 Jak to funguje?

Díky propojení **Snowflake Cortex** a **Streamlit** můžeme v reálném čase analyzovat textová data a převést je do vizuální podoby. Pomocí AI modelů se texty recenzí překládají, analyzují na sentiment a shrnují klíčové body.

V rámci této aplikace uvidíte různé interaktivní možnosti:
- **Filtrovat** recenze podle klíčových slov nebo sentimentu
- **Porovnávat** průměrné hodnocení poboček a sentiment
- **Vizualizovat** klíčová slova a jejich vztah k sentimentu a hodnocení

---

Pojďme se podívat na to, co nám naše data mohou prozradit! 👇


In [ ]:
CREATE OR REPLACE TABLE TEST.PUBLIC.ANALYZED_RECENZE_POBOCKY AS
SELECT 
    NAME,
    AUTHOR_RATING,
    REVIEW_TEXT,

    -- Překlad recenze do angličtiny pomocí COMPLETE a mistral-7b
    SNOWFLAKE.CORTEX.COMPLETE('mistral-7b', 'Translate the following Czech text to English accurately without adding or hallucinating any extra information: ' || REVIEW_TEXT) AS REVIEW_TEXT_ENGLISH,   

    -- Sentiment analýza přeloženého textu (nyní z přeloženého textu)
    SNOWFLAKE.CORTEX.SENTIMENT(REVIEW_TEXT_ENGLISH) AS SENTIMENT,

    -- Hlavní body generované z přeloženého textu
    SNOWFLAKE.CORTEX.COMPLETE('mistral-7b', 'Give one main keyword from this review, in a single word only, do not write more than one word in your response, give me just the keyword and nothing else: ' || REVIEW_TEXT_ENGLISH) AS KLICOVE_SLOVO
FROM 
    TEST.PUBLIC.RECENZE_POBOCKY;


In [ ]:
SELECT *
FROM TEST.PUBLIC.ANALYZED_RECENZE_POBOCKY

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Načtení dat z analyzované tabulky
df = cell5.to_pandas()

st.title("📊 Manažerský Dashboard: Analýza Recenzí Poboček")

# 1. Vyhledávání podle klíčových slov
st.subheader("🔍 Vyhledávání recenzí podle klíčových slov")
search_query = st.text_input("Zadejte slovo nebo frázi k vyhledání:")

if search_query:
    filtered_df = df[df['REVIEW_TEXT_ENGLISH'].str.contains(search_query, case=False) | df['KLICOVE_SLOVO'].str.contains(search_query, case=False)]
    st.write(f"**Nalezeno {len(filtered_df)} recenzí obsahujících '{search_query}':**")
    st.dataframe(filtered_df[['NAME', 'AUTHOR_RATING', 'REVIEW_TEXT', 'REVIEW_TEXT_ENGLISH', 'SENTIMENT', 'KLICOVE_SLOVO']])
else:
    st.info("Zadejte slovo nebo frázi pro vyhledávání, a prozkoumejte recenze poboček dle klíčových slov!")

# 2. Filtrování podle sentimentu s výraznými kategoriemi
st.subheader("🎭 Filtrovat recenze podle sentimentu")
sentiment_options = {
    "Velmi negativní (-1 až -0.5)": (-1, -0.5),
    "Negativní (-0.5 až 0)": (-0.5, 0),
    "Neutrální (0)": (0, 0),
    "Pozitivní (0 až 0.5)": (0, 0.5),
    "Velmi pozitivní (0.5 až 1)": (0.5, 1)
}
selected_sentiment = st.radio("Vyberte kategorii sentimentu:", list(sentiment_options.keys()))
min_sentiment, max_sentiment = sentiment_options[selected_sentiment]

filtered_by_sentiment = df[(df['SENTIMENT'] >= min_sentiment) & (df['SENTIMENT'] <= max_sentiment)]
st.write(f"Zobrazuji {len(filtered_by_sentiment)} recenzí s vybranou kategorií sentimentu **{selected_sentiment}**:")
st.dataframe(filtered_by_sentiment[['NAME', 'AUTHOR_RATING', 'REVIEW_TEXT', 'REVIEW_TEXT_ENGLISH', 'SENTIMENT', 'KLICOVE_SLOVO']])

# 3. Průměrné hodnocení poboček
st.subheader("⭐ Průměrné hodnocení poboček")
avg_rating_df = df.groupby('NAME')['AUTHOR_RATING'].mean().reset_index()
fig_avg_rating = px.bar(avg_rating_df, x='AUTHOR_RATING', y='NAME', orientation='h', color='AUTHOR_RATING',
                        color_continuous_scale='Viridis', title="Průměrné hodnocení poboček")
fig_avg_rating.update_layout(xaxis_title="Průměrné hodnocení", yaxis_title="Pobočka")
st.plotly_chart(fig_avg_rating)

# 4. Insight tabulka pro klíčová slova s podbarvením na základě sentimentu
st.subheader("📋 Insight Tabulka pro TOP 10 Kladných a Záporných Klíčových Slov")

# Zajištění, že se použije pouze první slovo klíčového slova
exploded_df = df.assign(KEYWORD=df['KLICOVE_SLOVO'].str.split().str[0]).dropna()
insight_df = exploded_df.groupby('KEYWORD').agg(
    prumerny_sentiment=('SENTIMENT', 'mean')
).reset_index()

# Rozdělení na kladná a záporná klíčová slova
top_positive_keywords = insight_df.nlargest(10, 'prumerny_sentiment').reset_index(drop=True)
top_negative_keywords = insight_df.nsmallest(10, 'prumerny_sentiment').reset_index(drop=True)

# Funkce pro jednoduché podbarvení na základě hodnot sentimentu
def color_sentiment(val):
    color = 'lightgreen' if val > 0 else 'lightcoral'
    return f'background-color: {color}'

# Zobrazení tabulek s podbarvením bez indexu
col1, col2 = st.columns(2)

with col1:
    st.markdown("**TOP 10 Kladných Klíčových Slov**")
    st.dataframe(top_positive_keywords.style.map(color_sentiment, subset=['prumerny_sentiment']).set_properties(**{'text-align': 'center'}))

with col2:
    st.markdown("**TOP 10 Záporných Klíčových Slov**")
    st.dataframe(top_negative_keywords.style.map(color_sentiment, subset=['prumerny_sentiment']).set_properties(**{'text-align': 'center'}))

# 5. Interaktivní Word Cloud pro kladné a záporné recenze z přeložených textů
st.subheader("🌥️ Interaktivní Word Cloud z přeložených textů recenzí")

# Nastavení pro Word Cloud
sentiment_choice = st.radio("Zobrazit Word Cloud pro:", ("Kladné Recenze", "Záporné Recenze"))
max_words = st.slider("Maximální počet slov:", min_value=10, max_value=200, value=100, step=10)

# Filtrace textů pro kladné nebo záporné recenze na základě výběru
if sentiment_choice == "Kladné Recenze":
    selected_texts = " ".join(df[df['SENTIMENT'] > 0.2]['REVIEW_TEXT_ENGLISH'].dropna())
    colormap = 'Greens'
else:
    selected_texts = " ".join(df[df['SENTIMENT'] < -0.2]['REVIEW_TEXT_ENGLISH'].dropna())
    colormap = 'Reds'

# Vytvoření a zobrazení Word Cloudu
wordcloud = WordCloud(background_color='white', colormap=colormap, width=800, height=400, max_words=max_words).generate(selected_texts)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
st.pyplot(plt)